In [ ]:
data_dir = "/data/projects/dschaub/ANCA-GN_transcriptomics/data/single-cell/exploratory/integrated_objects"
working_dir = "/data/projects/dschaub/ANCA-GN_transcriptomics"

In [ ]:
setwd(working_dir)
RhpcBLASctl::blas_set_num_threads(1)
library(Seurat)
library(dplyr)
#library(tidyverse)
library(here)
#library(readxl)
#library(Matrix)
library(IMSBSeuratHelpers)
library(ggplot2)
#library(sctransform)
library(future)
library(data.table)
library(xlsx)
#plan("multiprocess", workers = 20)
options(future.globals.maxSize = 20 * 1000 * 1024^2)#20GB
options(repr.matrix.max.rows=600, repr.matrix.max.cols=200)
library(harmony)

In [ ]:
samples_ID = "Harmony_ANCA_27PK"

In [ ]:
seu<-readRDS(paste0("integrated_objects/",samples_ID,"_all.rds"))
seu

In [ ]:
DimPlot(object = seu, reduction = 'umap',label = TRUE, 
        pt.size = 0.1,label.size = 6, repel = T) + theme(aspect.ratio=1)

In [ ]:
seu_new<-seu
umap = seu@reductions$umap@cell.embeddings
C.0 = colnames(seu_new)[umap[,1] > -5 & umap[,2] < 5]
C.1 = colnames(seu_new)[umap[,1] > 0 & umap[,2] > 5 & umap[,2] < 10]
C.2 = colnames(seu_new)[umap[,1] > 0.9 & umap[,2] > 10]
C.3 = colnames(seu_new)[umap[,1] > -5 & umap[,1] < 0.9 & umap[,2] > 10]
C.4 = colnames(seu_new)[umap[,1] > -10 & umap[,1] < -5 & umap[,2] > 5]
C.5 = colnames(seu_new)[umap[,1] < -7 & umap[,2] > -2.5 & umap[,2] < 5]
C.6 = colnames(seu_new)[umap[,1] < -11.3 & umap[,2] < -4.7]
C.7 <-setdiff(colnames(seu_new),c(C.0,C.1,C.2,C.3,C.4,C.5,C.6))
length(colnames(seu_new))
length(C.7)
length(c(C.0,C.1,C.2,C.3,C.4,C.5,C.6))

In [ ]:
Idents(seu_new, cells = C.0) <- "C.0"
Idents(seu_new, cells = C.1) <- "C.1"
Idents(seu_new, cells = C.2) <- "C.2"
Idents(seu_new, cells = C.3) <- "C.3"
Idents(seu_new, cells = C.4) <- "C.4"
Idents(seu_new, cells = C.5) <- "C.5"
Idents(seu_new, cells = C.6) <- "C.6"
Idents(seu_new, cells = C.7) <- "C.7"
DimPlot(object = seu_new, reduction = 'umap',label = TRUE, 
        pt.size = 0.1,label.size = 6, repel = T) + theme(aspect.ratio=1)

In [ ]:
levels(seu_new)<-c(
"C.0",
"C.1",
"C.2",
"C.3",
"C.4",
"C.5",
"C.6",
"C.7")
DimPlot(object = seu_new, reduction = 'umap',label = TRUE, 
        pt.size = 0.1,label.size = 6, repel = T) + theme(aspect.ratio=1)

In [ ]:
marker_qc<-c("nFeature_RNA","nCount_RNA","frac.mito","frac.ribo","CD3_count")
for (n in marker_qc){
    print(VlnPlot(object = seu_new, features = n,
                  #group.by = "patient",
                  #split.by = "cellgroup",
                  #x.lab.rot=T, 
                  #size.x.use = 5,
                  pt.size = 0
                       )#+ NoLegend()
    )

    print(FeaturePlot(object = seu_new,features = n,
                  cols = c("grey", "blue"),
                      order = T,
                      reduction = "umap",
                pt.size=0.1
                       ))     
    }

In [ ]:
# find markers for every cluster compared to all remaining cells, report only the positive ones
seu.markers <- FindAllMarkers(object = seu_new, only.pos = TRUE, 
                              min.pct = 0.25, logfc.threshold = 0.5)
#logfc.threshold=0.25 (default) instead of old version thresh.use=0.25
head(seu.markers)
dim(seu.markers)

In [ ]:
top3 <- seu.markers %>% group_by(cluster) %>% top_n(3, avg_log2FC)
top5 <- seu.markers %>% group_by(cluster) %>% top_n(5, avg_log2FC)
top10 <- seu.markers %>% group_by(cluster) %>% top_n(10, avg_log2FC)

In [ ]:
top10

In [ ]:
plt1<-DotPlot(seu_new, features = unique(top5$gene),
              dot.scale = 5
              #scale.by = "size"
             ) + coord_flip()+
theme(#strip.background = element_blank(),
    axis.title = element_blank(),
    axis.text.x = element_text(size = 12,angle = 45,hjust = 1),
    axis.text.y = element_text(size = 12),
     legend.position = "right",
     #legend.spacing = unit(0, "mm"),
     legend.direction = "vertical",
        legend.text = element_text(size=5),
        legend.key.width = unit(2, "mm"),
        legend.key.height = unit(2, "mm"),
        legend.box.spacing = unit(1, "mm"),
        legend.margin = margin(2),
        legend.title = element_text(size = 7,angle = 90)
)
plt1

In [ ]:
myeloid_markers_Covid<-c(
'CD68',
'MARCO',
'MRC1',
'APOE',
'SPP1',
'FABP5',
'CCL2',
'CCL3',
'FCN1',
'CD14',
'CD302',
'FCGR3A',
'TREM2',
'TMEM176A',
'CD86',
'TGFBI',
'HSPA6',
'HSPA1A',
'CCR7',
'SPINT2',
'LAMP3',
'FCGR3B',
'CXCR2',
'SPI1',
'CXCR4'
    )

In [ ]:
myeloid_markers_tianran<-c(
'S100A8',
'S100A9',
'SERPINB2',
'TNFAIP6',
'S100A12',
'RNASE1',
'LYVE1',
'SELENOP',
'FOLR2',
'EGFL7',
'FCGR3A',
'CDKN1C',
'SMIM25',
'LST1',
'SERPINA1',
'VCAN',
'LYZ',
'HLA-DQA1',
'CD1C',
'PKIB',
'CLEC10A',
'IL1R2',
'DNASE1L3',
'C1orf54',
'HLA-DPA1',
'HLA-DPB1',
'LRRC26',
'PTCRA',
'SCT',
'JCHAIN',
'CLEC4C',
'TPSD1',
'CPA3',
'TPSAB1',
'TPSB2',
'HDC'
)

In [ ]:
kidney_markers_DKD<-c(
"NPHS2",
"CDKN1C",
"WT1",
"PLAT",
"EHD3",
"EMCN",
"FABP4",
"PLVAP",
"RSAD2",
"CLDN5",
"PTPRB",
"VWF",
"ELN",
"GATA3",
#"HOPX"
#"PTN",
#"PIEZO2",
"ITGA8",
"ADAMTS5",
"REN1",
"ACTA2",
"TAGLN",
"SLC5A2",
"SLC5A12",
"GATM",
"SLC13A3",
"CYP4A10",
"SLC5A8",
#"SLC7A12",
#"CYP7B1",
#"SLC22A7",
#"SERPINA1F",
#"SLC22A13",
"AQP1",
"BST1",
"ANGPT2",
"SLC12A1",
"CLDN10",
"UMOD",
"SLC12A3",
"PVALB",
"CALB1",
"TMEM52B",
"SLC8A1",
"HSD11B2",
"AQP2",
"FXYD4",
"ATP6V1G3",
"ATP6V0D2",
"AQP6",
"SLC26A4")

In [ ]:
Epithelial_markers<-c(
"SCGB1A1",
"WFDC2",
"SCGB3A1")

In [ ]:
plt1<-DotPlot(seu_new, features = myeloid_markers_Covid,
              dot.scale = 5
              #scale.by = "size"
             ) + coord_flip()+
theme(#strip.background = element_blank(),
    axis.title = element_blank(),
    axis.text.x = element_text(size = 12,angle = 45,hjust = 1),
    axis.text.y = element_text(size = 12),
     legend.position = "right",
     #legend.spacing = unit(0, "mm"),
     legend.direction = "vertical",
        legend.text = element_text(size=5),
        legend.key.width = unit(2, "mm"),
        legend.key.height = unit(2, "mm"),
        legend.box.spacing = unit(1, "mm"),
        legend.margin = margin(2),
        legend.title = element_text(size = 7,angle = 90)
)
plt1

In [ ]:
plt1<-DotPlot(seu_new, features = myeloid_markers_tianran,
              dot.scale = 5
              #scale.by = "size"
             ) + coord_flip()+
theme(#strip.background = element_blank(),
    axis.title = element_blank(),
    axis.text.x = element_text(size = 12,angle = 45,hjust = 1),
    axis.text.y = element_text(size = 12),
     legend.position = "right",
     #legend.spacing = unit(0, "mm"),
     legend.direction = "vertical",
        legend.text = element_text(size=5),
        legend.key.width = unit(2, "mm"),
        legend.key.height = unit(2, "mm"),
        legend.box.spacing = unit(1, "mm"),
        legend.margin = margin(2),
        legend.title = element_text(size = 7,angle = 90)
)
plt1

In [ ]:
plt1<-DotPlot(seu_new, features = kidney_markers_DKD,
              dot.scale = 6
              #scale.by = "size"
             ) + coord_flip()+
theme(#strip.background = element_blank(),
    axis.title = element_blank(),
    axis.text.x = element_text(size = 12,angle = 45,hjust = 1),
    axis.text.y = element_text(size = 12),
     legend.position = "right",
     #legend.spacing = unit(0, "mm"),
     legend.direction = "vertical",
        legend.text = element_text(size=5),
        legend.key.width = unit(2, "mm"),
        legend.key.height = unit(2, "mm"),
        legend.box.spacing = unit(1, "mm"),
        legend.margin = margin(2),
        legend.title = element_text(size = 7,angle = 90)
)
plt1

In [ ]:
plt1<-DotPlot(seu_new, features = Epithelial_markers,
              dot.scale = 5
              #scale.by = "size"
             ) + coord_flip()+
theme(#strip.background = element_blank(),
    axis.title = element_blank(),
    axis.text.x = element_text(size = 12,angle = 45,hjust = 1),
    axis.text.y = element_text(size = 12),
     legend.position = "right",
     #legend.spacing = unit(0, "mm"),
     legend.direction = "vertical",
        legend.text = element_text(size=5),
        legend.key.width = unit(2, "mm"),
        legend.key.height = unit(2, "mm"),
        legend.box.spacing = unit(1, "mm"),
        legend.margin = margin(2),
        legend.title = element_text(size = 7,angle = 90)
)
plt1

In [ ]:
new.cluster.ids <- c(
'T cell',
'Proliferating',
'B cell',
'Macrophage',
'ATP_high1',
'PT',
'ATP_high2',
'DCT_PC'
)
names(new.cluster.ids) <- levels(seu_new)
seu_1 <- RenameIdents(seu_new, new.cluster.ids)

In [ ]:
DimPlot(object = seu_1, reduction = 'umap',label = TRUE, 
        pt.size = 0.1,label.size = 6, repel = T) + theme(aspect.ratio=1)

In [ ]:
seu_1@meta.data$cell_type<-Idents(seu_1)
head(seu_1@meta.data)

In [ ]:
seu_clean<-subset(seu_1, idents=c('ATP_high1','ATP_high2'),invert= T)
seu_clean
DimPlot(object = seu_clean, reduction = 'umap',label = TRUE, 
        pt.size = 0.1,label.size = 6, repel = T) + theme(aspect.ratio=1)


In [ ]:
Idents(seu_clean)<-"cell_type"

In [ ]:
DimPlot(object = seu_clean, reduction = 'umap',label = TRUE, 
        pt.size = 0.1,label.size = 6, repel = T) + theme(aspect.ratio=1)

In [ ]:
saveRDS(seu_clean, file = paste0(data_dir, "/Harmony_ANCA_27PK_annotated.rds"))